In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 channel, 6 output channels 5x5 conv kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # fully connected
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a 2x2 window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # 2 implies 2x2
        x = x.view(-1, self.n_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def n_flat_features(self, x):
        size = x.size()[1:] # ignore batch dimension
        n_features = 1
        for s in size:
            n_features *= s
        return n_features

In [15]:
net = Net()

In [16]:
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [17]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [18]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0324,  0.1110,  0.0507,  0.1061, -0.0214, -0.1076,  0.1157,
          0.1276, -0.0715, -0.0906]])


In [20]:
# Zero the gradient buffers of all params and backprop with rand gradients
net.zero_grad()
out.backward(torch.randn(1, 10))

# Loss Function

In [21]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(0.8424)


In [22]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

# Backprop

In [23]:
net.zero_grad()
print('conv1.bias.grad before backward():')
print(net.conv1.bias.grad)
loss.backward()

print('after:')
print(net.conv1.bias.grad)

conv1.bias.grad before backward():
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
after:
tensor(1.00000e-02 *
       [-0.0987,  0.5643, -1.2025,  0.5640,  0.1735,  0.1853])


# Updating weights

In [27]:
LEARNING_RATE = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * LEARNING_RATE)

In [28]:
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE)

In [29]:
# in training loop
optimizer.zero_grad ()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()